<div align="center">
  <h1>欢迎来到 ModelAssitant 的 Notebook 示例培训 🔥</h1>
  <a href="https://sensecraftma.seeed.cc/" target="_blank"><img width="20%" src="https://files.seeedstudio.com/sscma/docs/images/SSCMA-Hero.png"></a>
</div>

## ⚠️ 重要提示

在开始之前，请确保：

1. **启用 GPU 加速**：点击菜单栏 `运行时` → `更改运行时类型` → 在 `硬件加速器` 中选择 `GPU`
2. **Colab 版本要求**：**必须使用 Colab 2025.07 版本**以确保兼容性，其他版本可能会导致意外错误。

如果您遇到版本相关的问题，可以尝试在第一个代码单元格中运行以下命令来检查环境：

```python
# 检查 GPU 是否可用
!nvidia-smi

# 检查 Python 版本
!python --version
```

# Person Classification - MobileNetV2 0.35 Rep

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/seeed-studio/sscma-model-zoo/blob/main/notebooks/en/Person_Classification_MobileNetV2_0.35_Rep_64.ipynb)

**版本:** 1.0.0

**任务:** Image Classification

**算法:** [MobileNetV2 0.35 Rep](configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py)

**数据集:** [VWW](https://github.com/Mxbonn/visualwakewords)

**类别:** `Not a person`, `Person`

![Person Classification](https://files.seeedstudio.com/sscma/static/person_cls.png)

The model is a vision model designed for person classification. It utilizes the [SSCMA](https://github.com/Seeed-Studio/ModelAssistant) training and employs the MobileNetV2 (0.35) Rep algorithm.



## ⚙️准备工作
克隆 [仓库](https://github.com/Seeed-Studio/ModelAssistant) 并安装依赖项。
### 配置SSCMA

In [ ]:
!pip install ethos-u-vela
!git clone https://github.com/Seeed-Studio/ModelAssistant.git -b 2.0.0  #克隆仓库
%cd ModelAssistant
!. ./scripts/setup_colab.sh

### 下载模型权重文件

In [ ]:
%mkdir -p Person_Classification_MobileNetV2_0.35_Rep_64 
!wget -c https://files.seeedstudio.com/sscma/model_zoo/classification/person/mobilenetv2_0.35rep_vww64_float32_sha1_6dec3c029041408de043c5921621ab7abc4c4ec4.pth -O Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

### 下载数据集

In [ ]:
%mkdir -p Person_Classification_MobileNetV2_0.35_Rep_64/dataset 
!pip install roboflow
import os
from google.colab import userdata
os.environ["ROBOFLOW_API_KEY"] = userdata.get("ROBOFLOW_API_KEY")
from roboflow import download_dataset
dataset = download_dataset("https://github.com/Mxbonn/visualwakewords", "coco")
!mv dataset Person_Classification_MobileNetV2_0.35_Rep_64/dataset


## 🚀使用 SSCMA 训练模型
所有的训练参数都在 `config.py` 文件中，您可以更改这些参数以训练自己的模型。

下面是一些常见参数的解释。您还可以参考[文档](https://sensecraftma.seeed.cc/tutorials/config)获取更多详情。
- `data_root` - 数据集路径。
- `epochs` - 训练周期数。**这里为了方便演示，我们将其设置为 10**。
- `batch_size` - 批次大小。
- `height` - 图像高度。
- `width` - 图像宽度。
- `load_from` - 预训练模型路径。
- `num_classes` - 数据集类别数。

您可以使用 `--cfg-options` 参数覆盖 `config.py` 文件中的参数。
```bash
# 示例
sscma.train config.py --cfg-options data_root=./datasets/test_dataset epochs=10
```

In [ ]:
!sscma.train configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py \
--cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

## 📦导出模型
训练完成后，您可以将模型导出为部署所需的格式。目前 SSCMA 支持导出到 ONNX 和 TensorFlow Lite 格式。
您还可以参考[文档](https://sensecraftma.seeed.cc/tutorials/export/overview)获取更多详情。
```bash
python3 tools/export.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>" 
```

In [ ]:
import os
with open('Person_Classification_MobileNetV2_0.35_Rep_64/last_checkpoint', 'r') as f:
	os.environ['CHECKPOINT_FILE_PATH'] = f.read()

In [ ]:
!sscma.export configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py $CHECKPOINT_FILE_PATH --cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

## 📝评估模型
导出模型后，您可以使用 `inference.py` 脚本评估模型的性能。

```bash
python3 tools/inference.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>" 
```

### 评估 PyTorch 模型

In [ ]:
!sscma.inference configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py ${CHECKPOINT_FILE_PATH%.*}.pth \
--cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

### 评估 ONNX 模型

In [ ]:
!sscma.inference configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py ${CHECKPOINT_FILE_PATH%.*}_float32.onnx \
--cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

### 评估 TFLite Float32 模型

In [ ]:
!sscma.inference configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py ${CHECKPOINT_FILE_PATH%.*}_float32.tflite \
--cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

### 评估 TFLite INT8 模型

In [ ]:
!sscma.inference configs/classification/mobnetv2_0.35_rep_1bx16_300e_custom.py ${CHECKPOINT_FILE_PATH%.*}_int8.tflite \
--cfg-options  \
    work_dir=Person_Classification_MobileNetV2_0.35_Rep_64 \
    num_classes=2 \
    epochs=10  \
    height=64 \
    width=64 \
    data_root=Person_Classification_MobileNetV2_0.35_Rep_64/dataset/ \
    load_from=Person_Classification_MobileNetV2_0.35_Rep_64/pretrain.pth

## 🤖模型部署
在进行模型训练，评估和导出后，您可以将模型部署到您的设备上。您可以参考[文档](https://sensecraftma.seeed.cc/deploy/overview)获取更多详情。
你可以在下面的文件夹中获取所有的模型：

In [ ]:
%ls -lh Person_Classification_MobileNetV2_0.35_Rep_64/

## 感谢您尝试 SSCMA 🎉

恭喜您完成本教程。如果您对更多应用场景或我们的项目感兴趣，请在 GitHub 上给 [ModelAssistant](https://github.com/Seeed-Studio/ModelAssistant) 加个星 ✨。

如果您对本教程有任何问题，请随时[提交问题](https://github.com/Seeed-Studio/ModelAssistant/issues)。